In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *
import pandas as pd
import numpy as np
from itertools import cycle

In [ ]:
#| export
def foo(): pass

In [ ]:
#|export
def say_hello(to):
    "Say hello to somebody"
    return f'Hello {to}!'

In [ ]:
say_hello("water")

'Hello water!'

## test index

In [ ]:
from IPython.display import display,SVG

In [ ]:
display(SVG('<svg height="100" xmlns="http://www.w3.org/2000/svg"><circle cx="50" cy="50" r="40"/></svg>'))

In [ ]:
assert say_hello("Hamel")=="Hello Hamel!"

In [ ]:
say_hello("Isaac")

'Hello Isaac!'

## test dataframe


In [ ]:
#|export

def generate_fake_dataframe(size, cols, col_names = None, intervals = None, seed = None):
    'generate long fake dataframe'
    
    categories_dict = {'animals': ['cow', 'rabbit', 'duck', 'shrimp', 'pig', 'goat', 'crab', 'deer', 'bee', 'sheep', 'fish', 'turkey', 'dove', 'chicken', 'horse'],
                       'names'  : ['James', 'Mary', 'Robert', 'Patricia', 'John', 'Jennifer', 'Michael', 'Linda', 'William', 'Elizabeth', 'Ahmed', 'Barbara', 'Richard', 'Susan', 'Salomon', 'Juan Luis'],
                       'cities' : ['Stockholm', 'Denver', 'Moscow', 'Marseille', 'Palermo', 'Tokyo', 'Lisbon', 'Oslo', 'Nairobi', 'Río de Janeiro', 'Berlin', 'Bogotá', 'Manila', 'Madrid', 'Milwaukee'],
                       'colors' : ['red', 'orange', 'yellow', 'green', 'blue', 'indigo', 'purple', 'pink', 'silver', 'gold', 'beige', 'brown', 'grey', 'black', 'white']
                      }
    default_intervals = {"i" : (0,10), "f" : (0,100), "c" : ("names", 5), "d" : ("2020-01-01","2020-12-31")}
    rng = np.random.default_rng(seed)

    first_c = default_intervals["c"][0]
    categories_names = cycle([first_c] + [c for c in categories_dict.keys() if c != first_c])
    default_intervals["c"] = (categories_names, default_intervals["c"][1])
    
    if isinstance(col_names,list):
        assert len(col_names) == len(cols), f"The fake DataFrame should have {len(cols)} columns but col_names is a list with {len(col_names)} elements"
    elif col_names is None:
        suffix = {"c" : "cat", "i" : "int", "f" : "float", "d" : "date"}
        col_names = [f"column_{str(i)}_{suffix.get(col)}" for i, col in enumerate(cols)]

    if isinstance(intervals,list):
        assert len(intervals) == len(cols), f"The fake DataFrame should have {len(cols)} columns but intervals is a list with {len(intervals)} elements"
    else:
        if isinstance(intervals,dict):
            assert len(set(intervals.keys()) - set(default_intervals.keys())) == 0, f"The intervals parameter has invalid keys"
            default_intervals.update(intervals)
        intervals = [default_intervals[col] for col in cols]
    df = pd.DataFrame()
    for col, col_name, interval in zip(cols, col_names, intervals):
        if interval is None:
            interval = default_intervals[col]
        assert (len(interval) == 2 and isinstance(interval, tuple)) or isinstance(interval, list), f"This interval {interval} is neither a tuple of two elements nor a list of strings."
        if col in ("i","f","d"):
            start, end = interval
        if col == "i":
            df[col_name] = rng.integers(start, end, size)
        elif col == "f":
            df[col_name] = rng.uniform(start, end, size)
        elif col == "c":
            if isinstance(interval, list):
                categories = np.array(interval)
            else:
                cat_family, length = interval
                if isinstance(cat_family, cycle):
                    cat_family = next(cat_family)
                assert cat_family in categories_dict.keys(), f"There are no samples for category '{cat_family}'. Consider passing a list of samples or use one of the available categories: {categories_dict.keys()}"
                categories = rng.choice(categories_dict[cat_family], length, replace = False, shuffle = True)
            df[col_name] = rng.choice(categories, size, shuffle = True)
        elif col == "d":
            df[col_name] = rng.choice(pd.date_range(start, end), size)
    return df 

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcdccc")

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_float,column_6_int,column_7_cat,column_8_cat,column_9_date,column_10_cat,column_11_cat,column_12_cat
0,John,5,61.193342,4,68.426139,14.048826,7,cow,Milwaukee,2020-07-14,purple,Susan,bee
1,Robert,0,89.328390,6,57.972608,59.853675,6,pig,Milwaukee,2020-01-10,yellow,Susan,bee
2,James,4,83.378407,4,92.118090,1.811036,4,pig,Madrid,2020-02-17,red,Susan,crab
3,Robert,3,58.226111,2,21.439686,26.152232,0,cow,Río de Janeiro,2020-08-03,red,Ahmed,bee
4,Robert,2,70.871433,6,93.940500,59.478504,3,pig,Milwaukee,2020-01-19,red,Susan,rabbit
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Robert,9,63.315822,5,95.211791,12.504884,9,rabbit,Milwaukee,2020-08-25,grey,Robert,fish
996,Jennifer,6,15.849040,7,5.426221,86.116322,2,cow,Moscow,2020-01-30,silver,Ahmed,rabbit
997,Robert,6,95.940399,0,0.237634,61.325197,8,pig,Manila,2020-11-08,yellow,Ahmed,bee
998,John,4,15.000606,2,46.333072,58.790997,1,pig,Milwaukee,2020-04-03,grey,Linda,crab


testing text with long dataframes

In [ ]:
generate_fake_dataframe(size = 1000, cols =  "cififficcdccdddcffic")

,column_0_cat,column_1_int,column_2_float,column_3_int,column_4_float,column_5_float,column_6_int,column_7_cat,column_8_cat,column_9_date,column_10_cat,column_11_cat,column_12_date,column_13_date,column_14_date,column_15_cat,column_16_float,column_17_float,column_18_int,column_19_cat
0,Barbara,2,0.592698,1,53.948799,82.693921,2,bee,Nairobi,2020-09-18,red,Linda,2020-03-30,2020-08-29,2020-08-11,fish,68.858510,37.669108,3,Madrid
1,Ahmed,2,89.360910,3,56.132189,82.141987,1,cow,Stockholm,2020-01-15,red,Elizabeth,2020-03-28,2020-06-25,2020-01-06,goat,85.357199,37.314934,4,Nairobi
2,Ahmed,2,88.085513,6,40.995277,1.238760,9,rabbit,Oslo,2020-07-31,red,James,2020-06-28,2020-05-07,2020-10-24,bee,63.668362,91.024442,7,Madrid
3,John,2,15.271805,1,7.774838,32.548798,0,cow,Stockholm,2020-02-22,red,Juan Luis,2020-04-26,2020-10-07,2020-06-27,goat,70.562731,27.319760,5,Denver
4,Ahmed,1,76.633678,1,43.738530,53.414830,1,cow,Stockholm,2020-11-22,orange,Elizabeth,2020-11-12,2020-10-03,2020-06-11,goat,50.854821,41.159733,8,Bogotá
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Jennifer,9,19.445782,0,64.100199,36.309334,7,goat,Stockholm,2020-11-24,orange,Barbara,2020-03-01,2020-03-14,2020-02-04,deer,40.249892,93.686079,9,Denver
996,Barbara,1,62.526543,5,40.538032,75.419391,6,goat,Oslo,2020-06-27,purple,Juan Luis,2020-12-09,2020-04-21,2020-09-13,dove,8.808362,97.661611,5,Denver
997,Jennifer,9,18.137012,2,86.524860,57.805289,1,cow,Río de Janeiro,2020-11-27,yellow,Linda,2020-02-11,2020-07-06,2020-01-31,fish,54.787747,82.193398,5,Nairobi
998,Jennifer,0,47.102823,5,42.539334,11.422088,0,bee,Nairobi,2020-07-13,red,Juan Luis,2020-12-25,2020-04-10,2020-02-15,fish,39.096335,87.683426,3,Nairobi


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()